# Оптимизация выполнения кода, векторизация, Numba

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. "Оптимизация выполнения кода, векторизация, Numba"
* IPython Cookbook, Second Edition (2018), глава 4
* https://ipython-books.github.io/43-profiling-your-code-line-by-line-with-line_profiler/
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html
* https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

## Задачи для совместного разбора

In [ ]:
!pip install line_profiler

In [ ]:
import numpy as np
import numba
from numba import njit

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [ ]:
N = 1_000_000
A = np.random.randint(0, 1000, N)
B = A + 100
np.mean(B), np.mean(A)

(599.285937, 499.285937)

2. Напишите функцию, которая возвращает сумму всех чисел от 0 до x-1. Примените функцию к каждому элементу массива.

In [ ]:
def sum(x):
    return np.sum(np.arange(x))
sum_v = np.vectorize(sum)
sum_v(B)

array([235641,  59685, 431985, ..., 151525,  51360, 183921])

3. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [ ]:
%load_ext line_profiler

In [ ]:
import pandas as pd
import string

N = 2000000
df = pd.DataFrame(np.random.randn(N, 4), columns=[f"col{i}" for i in range(4)])
df["key"] = np.random.choice(list(string.ascii_letters.lower()), N, replace=True)
df.head(2)

,col0,col1,col2,col3,key
0,-0.882918,0.103939,0.460958,-2.375668,s
1,2.213133,0.075920,-0.823871,0.712522,q


In [ ]:
def key_ae(key):
    return key in "abcde"
key_ae_vec = np.vectorize(key_ae)
df[key_ae_vec(df.key)]

,col0,col1,col2,col3,key
2,-1.674081,1.077885,1.205773,-0.305679,b
10,-0.919218,0.976939,-0.620981,0.544139,d
17,-1.079255,0.189474,0.247230,-0.217548,d
24,-0.911701,-0.110187,-0.395518,-0.595514,c
25,1.169336,0.481727,-0.751359,0.744799,d
...,...,...,...,...,...
1999967,-0.510435,-2.204288,-1.991320,-0.120440,d
1999968,-0.981010,2.142351,-0.689206,-1.067534,a
1999993,0.550008,-0.139919,-1.056733,-1.900924,a
1999995,0.518024,-0.271586,-0.080850,0.360681,a


## Лабораторная работа 3

In [ ]:
import numpy as np
import pandas as pd
from numba import jit, njit
import numba
from typing import Union
import re
import json

В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

## Измерение времени выполнения кода

Назовем полным описанием рецепта строку, полученную путем конкатенации названия и описания рецепта через пробел. Удалите строки для рецептов, которые были добавлены не в 2010 году.

Реализуйте несколько вариантов функции подсчета средней длины полного описания рецепта для рецептов, добавленных в 2010 году.

In [ ]:
recipes = pd.read_csv("/content/recipes_sample.csv", parse_dates=["submitted"])
reviews = pd.read_csv("/content/reviews_sample.csv", index_col=0, parse_dates=["date"])

In [ ]:
recipes_2010 = recipes[recipes.submitted.dt.year==2010]
recipes_2010.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
52,just peachy cobbler,437637,70,1085867,2010-09-17,10.0,all i can say is yummmmmm . . . a simple to ma...,10.0
68,the heat spicy party mix,437219,95,1682162,2010-09-13,NaN,a spicy chex mix that will really warm your gu...,11.0
81,iowa state fair sweet dough caramel cinnamon ...,435816,80,17803,2010-08-24,29.0,this was the winning entry at the 2010 iowa st...,NaN
104,1 minute blueberries cream,428566,2,1375473,2010-06-04,4.0,i was craving blueberry tonight but wanted non...,NaN
146,2 2 2 diet mocha,416599,5,789314,2010-03-15,5.0,"while trying to come up with a satisfying ""sna...",7.0


1\.1 С использованием метода `DataFrame.iterrows` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - нахождение полного описания рецепта осуществляется внутри цикла по `iterrows` для каждой строки по отдельности.

In [ ]:
def task1_1(df: pd.DataFrame) -> float:
    temp = np.empty(0)
    for id_, row in df.iterrows():
        temp = np.append(temp, len(f"{row['name']} {row.description}"))
        
    return np.sum(temp)/temp.shape[0]

In [ ]:
task1_1(recipes_2010)

265.501300390117

1\.2. С использованием метода `DataFrame.apply` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - вызываете метод apply у таблицы, в качестве аргумента передаете функцию, которая возвращает полное описание для каждой строки;
    
    - считаете среднюю длину описаний, вызвав соответствующий метод серии.

In [ ]:
def task1_2(df: pd.DataFrame) -> float:
    return df.apply(lambda row: f"{row['name']} {row.description}", axis=1).str.len().mean()

In [ ]:
task1_2(recipes_2010)

265.501300390117

1\.3. С использованием векторизованных методов серий `pd.Series`:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - при помощи векторизированных операций получаете столбец с полным описанием;
    
    - при помощи векторизированных операций получаете длины полного описания;
    
    - при помощи метода серий получаете среднюю длину полных описаний. 

In [ ]:
def task1_3(df: pd.DataFrame) -> float:
    full_names = df['name'] + ' ' + df.description
    vec_len = np.vectorize(lambda row: len(row))
    lens = vec_len(full_names)
    
    return lens.mean()

In [ ]:
task1_3(recipes_2010)

265.501300390117

1.4 Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций при помощи магических команд `time` и `timeit`.

In [ ]:
%time task1_1(recipes_2010)
%time task1_2(recipes_2010)
%time task1_3(recipes_2010)

CPU times: user 116 ms, sys: 1.62 ms, total: 117 ms
Wall time: 121 ms
CPU times: user 33.4 ms, sys: 0 ns, total: 33.4 ms
Wall time: 33.9 ms
CPU times: user 1.91 ms, sys: 0 ns, total: 1.91 ms
Wall time: 1.73 ms


265.501300390117

In [ ]:
%timeit task1_1(recipes_2010)
%timeit task1_2(recipes_2010)
%timeit task1_3(recipes_2010)

131 ms ± 7.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
31.8 ms ± 739 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.21 ms ± 104 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Анализ пошагового выполнения кода 

Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. 

In [ ]:
%%writefile mprun_for.py

import re


def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        recipe_id, review = row["recipe_id"], row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)

    word_reviews_count = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        review = row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

Writing mprun_for.py


In [ ]:
from mprun_for import get_word_reviews_count

In [ ]:
reviews.head()

,user_id,recipe_id,date,rating,review
370476,2.175200e+04,57993.0,2003-05-01,5.0,Last week whole sides of frozen salmon fillet ...
624300,4.318130e+05,142201.0,2007-09-16,5.0,So simple and so tasty! I used a yellow capsi...
187037,4.007080e+05,252013.0,2008-01-10,4.0,"Very nice breakfast HH, easy to make and yummy..."
706134,2.001852e+09,404716.0,2017-12-11,5.0,These are a favorite for the holidays and so e...
312179,9.581000e+04,129396.0,2008-03-14,5.0,Excellent soup! The tomato flavor is just gre...


2.1 Найдите узкие места в коде, проанализировав код функции по шагам, используя профайлер. Сохраните результаты работы профайлера в отдельную текстовую ячейку. Выпишите (словами), что в имеющемся коде реализовано неоптимально. 

In [ ]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

In [ ]:
# Timer unit: 1e-07 s

# Total time: 30.7155 s
# File: C:\Users\Nikita\Desktop\ShaRinGan\TOBD\8\mprun_for.py
# Function: get_word_reviews_count at line 5

# Line #      Hits         Time  Per Hit   % Time  Line Contents
# ==============================================================
#      5                                           def get_word_reviews_count(df):
#      6         1          5.0      5.0      0.0      word_reviews = {}
#      7    126679   78634014.0    620.7     25.6      for _, row in df.dropna(subset=["review"]).iterrows():
#      8    126679   26389886.0    208.3      8.6          recipe_id, review = row["recipe_id"], row["review"]
#      9    126679   10004130.0     79.0      3.3          words = re.sub("[^A-Za-z\s]", "", review).split(" ")
#     10   6792010   11979487.0      1.8      3.9          for word in words:
#     11   6698951   16224219.0      2.4      5.3              if word not in word_reviews:
#     12     93059     283698.0      3.0      0.1                  word_reviews[word] = []
#     13   6792010   22535409.0      3.3      7.3              word_reviews[word].append(recipe_id)
#     14                                           
#     15         1          2.0      2.0      0.0      word_reviews_count = {}
#     16    126679   75858812.0    598.8     24.7      for _, row in df.dropna(subset=["review"]).iterrows():
#     17    126679   14684557.0    115.9      4.8          review = row["review"]
#     18    126679   10055020.0     79.4      3.3          words = re.sub("[^A-Za-z\s]", "", review).split(" ")
#     19   6792010   12083794.0      1.8      3.9          for word in words:
#     20   6792010   28422129.0      4.2      9.3              word_reviews_count[word] = len(word_reviews[word])
#     21         1          3.0      3.0      0.0      return word_reviews_count

работа со списком занимает много времени, как и итерация циклов построчно

2.2  Оптимизируйте функцию и добейтесь значительного (как минимум, в 5 раз) прироста в скорости выполнения. Для демонстрации результата измерьте скорость выполнения оригинальной функции и функции, написанной вами.

In [ ]:
reviews.review[370476]

"Last week whole sides of frozen salmon fillet was on sale in my local supermarket, so I bought tons (okay, only 3, but total weight was over 10 pounds).  This recipe is perfect for salmon fillet, even though it calls for salmon steaks.  I cut up the salmon into individual portions and followed the instructions exactly.  I'm on one of those food combining diets, so I left out the white wine but added just a dash of white wine vinegar instead (just a little bit, not enough to change the taste of the dish).  Super yummy, and leftovers for lunch today (lucky me)!"

In [ ]:
def task2_2(df):
    df = df.dropna(subset=["review"])
    reviews_s = df.review.apply(lambda s: re.sub("[^A-Za-z\s]", "", s).split(" "))
    word_reviews_count = {}
    for review in reviews_s:
        for word in review:
            if word not in word_reviews_count:
                word_reviews_count[word] = 0
            word_reviews_count[word]+=1
            
    return word_reviews_count

In [ ]:
%timeit task2_2(reviews)
%timeit task2_2(reviews)

558 ms ± 68.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
504 ms ± 11.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Numba

В файле `rating_predictions.json` хранятся данные о рейтингах рецептов и прогнозных значениях рейтингов для этого рецепта, полученных при помощи модели машинного обучения. 

Напишите несколько версий функции (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от прогнозного значения рейтинга для данного рецепта. 


Замечание 1: в формуле MAPE под $A_t$ понимается рейтинг из отзыва $t$, под $F_t$ - прогнозное значения рейтинга отзыва $t$.

Замечание 2: в результате работы функций должно получиться одно число - MAPE для всего набора данных.

In [ ]:
A_list = []
F_list = []

with open("/content/rating_predictions.json", "r") as file:
  gg = json.load(file)
  
for g in gg:
  A_list.append(g["rating"])
  F_list.append(g['prediction'])

3\.1 Создайте два списка `A_list` и `F_list` на основе файла `rating_predictions.json`. Напишите функцию `mape_lists` без использования векторизованных операций и методов массивов `numpy` и без использования `numba` (проитерируйтесь по спискам и вычислите суммарное значение MAPE для всех элементов, а потом усредните результат).

Измерьте время выполнения данной функции на входных данных `A_list` и `F_list`. Временем, затрачиваемым на создание списков, можно пренебречь.
    

In [ ]:
def task3_1(A, F):
    res = 0
    for a, f in zip(A,F):
        res += abs((a-f)/a)
        
    return res/len(A)

In [ ]:
task3_1(A_list, F_list)

0.13325265503992637

In [ ]:
%timeit task3_1(A_list, F_list)

19.5 ms ± 2.55 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


3\.2. Создайте массивы `numpy` `A_array` и `F_array` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numpy` с использованием векторизованных операций и методов массивов `numpy`.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`. Временем, затрачиваемым на создание массивов, можно пренебречь.

In [ ]:
A_array = np.array(A_list)
F_array = np.array(F_list)

def task3_2(A, F):
    return np.abs((A-F)/A).mean()  

In [ ]:
task3_2(A_array, F_array)

0.13325265503991449

In [ ]:
%timeit task3_2(A_array, F_array)

569 µs ± 73 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


3\.3. Создайте объекты `numba.typed.List` `A_typed` и `F_typed` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numba` без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`. 

Измерьте время выполнения данной функции на входных данных `A_typed` и `F_typed`. Временем, затрачиваемым на создание объектов `numba.typed.List`, можно пренебречь.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`.

In [ ]:
A_typed = numba.typed.List(A_list)
F_typed = numba.typed.List(F_list)

@njit
def task3_3(A, F):
    res = 0
    for a, f in zip(A,F):
        res += abs((a-f)/a)
        
    return res/len(A)

In [ ]:
%timeit task3_3(A_typed, F_typed)

1.11 ms ± 70.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Векторизация

Сайт-агрегатор устроил акцию: он дарит купоны на посещение ресторана тем пользователям, оставившим отзывы, идентификатор которых является _красивым числом_. Натуральное число называется _красивым_, если первая цифра числа совпадает с последней цифрой числа. 



4\.1 Напишите функцию `is_pretty`, которая для каждого идентификатора пользователя из файла определяет, получит ли он подарок. Запрещается преобразовывать идентификатор пользователя к строке. Подтвердите корректность реализации, продемонстрировав примеры.

In [ ]:
ids = reviews["recipe_id"].values

In [ ]:
def is_pretty(n: int) -> bool:
    last = n % 10
    while n >= 10:
        n //= 10
        
    return n == last

In [ ]:
is_pretty(54333215), is_pretty(14333215)

(True, False)

4\.2 Посчитайте с помощью функции `is_pretty` количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.

In [ ]:
count = 0
for id in ids:
    if is_pretty(id):
        count += 1
count

11955

In [ ]:
%%timeit
count = 0
for id in ids:
    if is_pretty(id):
        count += 1

168 ms ± 1.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


4\.3. При помощи `numpy` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [ ]:
is_pretty_vec = np.vectorize(is_pretty)

In [ ]:
np.sum(is_pretty_vec(ids))

11955

In [ ]:
%%timeit
np.sum(is_pretty_vec(ids))

37.1 ms ± 184 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


4\.4. При помощи `numba` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [ ]:
@njit
def is_pretty_numba(arr):
    count = 0
    for n in arr:
        last = n % 10
        while n >= 10:
            n //= 10
        if n == last:
            count += 1

    return count

In [ ]:
is_pretty_numba(ids)

11955

In [ ]:
%%timeit
is_pretty_numba(ids)

792 µs ± 243 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
